In [13]:
import sys
sys.path.append('../..')
import acquire_ear_dataset as a
import torch
import numpy as np
from PIL import Image
import glob
from torch import cuda
import os
import shutil
from DLBio.pytorch_helpers import get_device
from os.path import join
import ds_ear_siamese as des
import transforms_data as td
from torch.autograd import Variable
import torch.nn.functional as F

NN_SIAMESE = True
AUTHORIZED = ["falco_len","konrad_von"]
DEVICE = get_device()
model = torch.load('./models/model.pt', DEVICE)

In [14]:
#Output for NN  
if NN_SIAMESE == True:
    def generate_output( img_in, img_in2):
        if cuda.is_available():
            return model(Variable(img_in).cuda(), Variable(img_in2).cuda())
        else:
            return model(Variable(img_in), Variable(img_in2))

# Output for Mobilenet
if NN_SIAMESE == False:
    def generate_output( img_in):
        if cuda.is_available():
            return model(Variable(img_in).cuda())
        else:
            return model(Variable(img_in))    

In [16]:
#a.capture_ear_images(amount_pic=10, pic_per_stage=10, is_authentification=True)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [15]:
# method for displaying files with index
def print_list(list_):
    fmt = '{:<8}{:<20}'
    print(fmt.format('Index', 'Name'))
    for i, name in enumerate(list_):
        print(fmt.format(i, name))

In [5]:
# takes the name input and returns full path to user
# if no name is selected, a list to choose from is presented
def choose_user(dataset_path, name='unknown'):
    users = os.listdir(dataset_path)
    user_name = name
    # if statement starts, when no name was given
    if user_name == 'unknown':
        print_list(users)
        # Handle the user's input for user name
        while True:
            idx = input('Choose your user name by index: ')
            try:
                idx = int(idx)
                assert idx < len(users) and idx >= 0
                break
            except (ValueError, AssertionError):
                print('The input was a string or not in the index range.')
        user_name = users[idx]
    
    assert user_name in users, 'The name was not found in the dataset.'
    
    return user_name

In [6]:
# Image processing
# input is the filename and the transforamtion function
def image_pipeline(input_, preprocess):
    input_ = Image.open(input_)
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, 100, 100, 1)
    input_ = input_.permute(3, 0, 1, 2)
    

    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [7]:
# Setting up the dataset
def get_triplets(dataset_path, user_name, verif_dataset):
    NUM_CLASSES = len(os.listdir( dataset_path ))
    user_imgs = os.listdir(join(dataset_path, user_name))
    
    preprocess = td.transforms_siamese_verification( (100, 100) )

    # Triplets list will contain anchor(A), positive(P) and negative(N) triplets.
    triplets = []
    # assert len(os.listdir(verif_dataset)) < len(os.listdir(user_path)), 'There are more verification images than po'
    # creating anchor, positive, negative triplets (amount equals taken images)
    for img_dir in os.listdir( verif_dataset ):
        anchor_dir = join(verif_dataset, img_dir)
        anchor = image_pipeline(anchor_dir, preprocess)

        # choose random image from the alleged user and pass the preprocess strategy
        positive = image_pipeline( join(dataset_path, user_name, user_imgs[np.random.randint(0, len(user_imgs))]), preprocess )
        
        # find a class different from anchor. if same as anchor try again
        neg_class = os.listdir(dataset_path)[np.random.randint(NUM_CLASSES)]
        while neg_class == anchor_dir: 
            neg_class = os.listdir(dataset_path)[np.random.randint(NUM_CLASSES)]
        # list of image file names
        neg_dir_imgs = os.listdir(join(dataset_path, neg_class))
        # choose random image from the negative and pass the preprocess strategy
        negative = image_pipeline( join(dataset_path, neg_class, neg_dir_imgs[np.random.randint(0, len(neg_dir_imgs))]), preprocess )

        # append triplet
        triplets.append([anchor, positive, negative])

    return triplets

In [8]:
# Funktionsaufruf Konrad
# pers_to_ver = input('\n Enter name end press <return> ==>  ')
pers_to_ver = choose_user('../dataset_low_res')
triplet_list = get_triplets(dataset_path='../dataset_low_res',user_name=pers_to_ver,
                    verif_dataset='../auth_dataset/unknown-auth')

Index   Name                
0       nils_loo            
1       sarah_feh           
2       johannes_boe        
3       konrad_von          
4       tim_moe             
5       mila_wol            
6       janna_qua           
7       alissa_buh          
8       falco_len           
9       jesse_kru           
10      johannes_wie        


In [12]:
verification_counter = 0
#Scaling factor
a = 0.1
for t in triplet_list:
    if NN_SIAMESE == True:
        match_out1, match_out2 = generate_output(t[0], t[1])
        non_match_out1, non_match_out2 = generate_output(t[0], t[2])
    if NN_SIAMESE == False:
        match_out1 = generate_output(t[0])
        match_out2 = generate_output(t[1])                 
        non_match_out1 = generate_output(t[0])
        non_match_out2 = generate_output(t[2])

    euclidean_distance_pp = F.pairwise_distance(match_out1, match_out2)
    euclidean_distance_pn = F.pairwise_distance(non_match_out1, non_match_out2)

    if(euclidean_distance_pp + a < euclidean_distance_pn): verification_counter += 1

    # format variables
    fmt_id = '{:<12}'
    fmt_eucl = '{:<.3f}'
    print(fmt_id.format('pos-pos: '), fmt_eucl.format( euclidean_distance_pp.item()) )
    print(fmt_id.format('pos-neg: '),fmt_eucl.format( euclidean_distance_pn.item()) )
    print(fmt_id.format('Acc. count: '), '{:>.0f}'.format(verification_counter), '\n')

TypeError: forward() takes 2 positional arguments but 3 were given

In [10]:
NUMBER_AUTHORIZED = int(.9*len(triplet_list))
if  pers_to_ver in AUTHORIZED and verification_counter >= NUMBER_AUTHORIZED:
    print("Access granted! Welcome "  + pers_to_ver + "!")
else:
    print("Access denied")

Access denied
